# append_paged_kv_cache 测试

**功能**: 把新生成的 token 的 Key/Value 写入到分页管理的 KV Cache 中

**应用场景**: 
- 在 LLM 推理的 decode 阶段，每生成一个新 token，都需要把它的 K/V 追加到 cache 中
- 使用分页管理避免连续内存分配，提高内存利用率（类似操作系统的虚拟内存）

In [1]:
import torch
import flashinfer

## 1. 配置参数

In [2]:
num_kv_heads = 4        # KV 头数（GQA: 每个 KV 头可以服务多个 Q 头）
head_dim = 128          # 每个头的维度
page_size = 16          # 每页存储的 token 数（类似 OS 的页大小）
device = "cuda:0"
dtype = torch.float16

print(f"✅ 配置: {num_kv_heads} KV heads, head_dim={head_dim}, page_size={page_size}")

✅ 配置: 4 KV heads, head_dim=128, page_size=16


## 2. 场景设置

**模拟场景**: 已有 4096 个 token，现在追加第 4097 个

- 4096 个 token 需要 256 页（每页 16 个 token）
- 追加 1 个 token 后，需要第 257 页（因为第 256 页已满）

In [3]:
current_seq_len = 4096      # 当前已缓存的序列长度
nnz_kv = 1                  # 要追加的 token 数量（通常是 1）

# 计算需要的总页数（向上取整）
total_seq_len = current_seq_len + nnz_kv
num_pages_needed = (total_seq_len + page_size - 1) // page_size

print(f"当前序列长度: {current_seq_len}")
print(f"追加 token 数: {nnz_kv}")
print(f"总序列长度: {total_seq_len}")
print(f"📄 需要的总页数: {num_pages_needed} (= ceil({total_seq_len} / {page_size}))")

当前序列长度: 4096
追加 token 数: 1
总序列长度: 4097
📄 需要的总页数: 257 (= ceil(4097 / 16))


## 3. 创建 Page Pool 和分配页

**Page Pool**: 全局共享的页池（类似物理内存），所有请求共享这个池子

**页分配策略**: 随机分配页（模拟内存碎片化），实际推理中页分布不连续

In [4]:
max_num_pages = 60000  # 页池总容量（所有请求共享）

# 创建全局页池: [页数, K/V, 页内token数, KV头数, 头维度]
paged_kv_cache = torch.empty(
    max_num_pages,      # 60000 页
    2,                  # 维度0: K, 维度1: V
    page_size,          # 16 个 token/页
    num_kv_heads,       # 4 个 KV 头
    head_dim,           # 128 维
    device=device, dtype=dtype
)

# 随机分配页（模拟内存碎片化，实际推理中页分布不连续）
all_indices = torch.randperm(max_num_pages, device=device, dtype=torch.int32)
kv_page_indices = all_indices[:num_pages_needed]  # 为这个请求分配 257 个不连续的页

print(f"📦 paged_kv_cache shape: {paged_kv_cache.shape}")
print(f"📊 总内存占用: {paged_kv_cache.element_size() * paged_kv_cache.numel() / 1024**3:.2f} GB")
print(f"🎲 分配的页索引 (前10个): {kv_page_indices[:10].tolist()}")

📦 paged_kv_cache shape: torch.Size([60000, 2, 16, 4, 128])
📊 总内存占用: 1.83 GB
🎲 分配的页索引 (前10个): [30411, 45369, 58963, 33828, 7349, 39171, 48704, 24206, 2923, 24479]


## 4. 构造元数据

**页表元数据** (类似进程的页表):
- `kv_page_indptr`: 标记每个请求使用的页索引范围
- `kv_last_page_len`: 最后一页当前的填充长度

In [5]:
# kv_page_indptr: 标记每个请求使用的页索引范围
# [0, 257] 表示请求 0 使用 kv_page_indices[0:257]
kv_page_indptr = torch.tensor([0, num_pages_needed], dtype=torch.int32, device=device)

# kv_last_page_len: 最后一页当前的填充长度
# 0 表示最后一页（Page 255）已满，新 token 会写入到新页（Page 256）的位置 0
kv_last_page_len = torch.tensor([0], dtype=torch.int32, device=device)

# 构造要追加的 K/V 数据 (随机生成，实际推理中来自 transformer 输出)
k_append = torch.randn(nnz_kv, num_kv_heads, head_dim, device=device, dtype=dtype)
v_append = torch.randn(nnz_kv, num_kv_heads, head_dim, device=device, dtype=dtype)

print(f"📌 kv_page_indptr: {kv_page_indptr.tolist()} (请求 0 使用页范围 [0, {num_pages_needed}))")
print(f"📌 kv_last_page_len: {kv_last_page_len.item()} (最后一页已满，从新页开始)")
print(f"📥 k_append shape: {k_append.shape} {list(k_append.shape)}")
print(f"📥 v_append shape: {v_append.shape} {list(v_append.shape)}")

📌 kv_page_indptr: [0, 257] (请求 0 使用页范围 [0, 257))
📌 kv_last_page_len: 0 (最后一页已满，从新页开始)
📥 k_append shape: torch.Size([1, 4, 128]) [1, 4, 128]
📥 v_append shape: torch.Size([1, 4, 128]) [1, 4, 128]


## 5. 生成位置信息

**关键步骤**: 计算每个新 token 应该写入的位置（序列位置，而非物理页号）

- `seq_lens`: 计算当前序列的实际长度
- `batch_indices`: 每个 token 属于哪个请求
- `positions`: 每个 token 在序列中的位置（0-indexed）

In [6]:
# kv_append_indptr: 标记每个请求要追加多少个 token
# [0, 1] 表示请求 0 追加 1 个 token
kv_append_indptr = torch.tensor([0, nnz_kv], dtype=torch.int32, device=device)

# 计算当前序列长度（基于页表和 last_page_len）
seq_lens = flashinfer.get_seq_lens(kv_page_indptr, kv_last_page_len, page_size)
print(f"📏 当前序列长度 (seq_lens): {seq_lens.tolist()}")

# 生成批次索引和写入位置
# batch_indices: 每个 token 属于哪个请求 (这里都是 0)
# positions: 每个 token 在序列中的位置 (这里是 4095，因为 0-indexed)
batch_indices, positions = flashinfer.get_batch_indices_positions(
    kv_append_indptr, seq_lens, nnz_kv
)

print(f"🎯 batch_indices: {batch_indices.tolist()} (属于请求 0)")
print(f"🎯 positions: {positions.tolist()} (写入位置，0-indexed)")
print(f"\n💡 解释: position {positions[0].item()} = 第 {positions[0].item() // page_size} 页 (page offset) + 第 {positions[0].item() % page_size} 个位置 (within-page offset)")

📏 当前序列长度 (seq_lens): [4096]


🎯 batch_indices: [0] (属于请求 0)
🎯 positions: [4095] (写入位置，0-indexed)

💡 解释: position 4095 = 第 255 页 (page offset) + 第 15 个位置 (within-page offset)


## 6. 执行追加

**核心操作**: 调用 `append_paged_kv_cache` 把新 token 的 K/V 写入到对应的页中

内部会根据 `positions` 计算页号和页内偏移，执行写入操作

In [7]:
# 核心操作：把新 token 的 K/V 写入到对应的页中
flashinfer.page.append_paged_kv_cache(
    k_append,           # [1, 4, 128] - 要追加的 K
    v_append,           # [1, 4, 128] - 要追加的 V
    batch_indices,      # [0] - 属于请求 0
    positions,          # [4095] - 写入位置（0-indexed）
    paged_kv_cache,     # [60000, 2, 16, 4, 128] - 全局页池
    kv_page_indices,    # [257] - 这个请求的页号列表
    kv_page_indptr,     # [0, 257] - 页索引范围
    kv_last_page_len    # [0] - 最后一页长度
)
print("✅ 追加完成！")

✅ 追加完成！


## 7. 验证结果

**验证逻辑**: 
1. 根据 `positions` 计算物理页号和页内偏移
2. 从 `paged_kv_cache` 中读取写入的数据
3. 与原始 `k_append` 比较，验证写入是否正确

In [8]:
# 计算物理页号和页内偏移
actual_pos = positions[0].item()              # 序列位置（4095）
page_offset = actual_pos // page_size         # 页号（255）
within_page_offset = actual_pos % page_size   # 页内偏移（15）

# 获取实际的物理页 ID（从页表中查找）
actual_page_id = kv_page_indices[page_offset]

# 从 cache 中读取写入的 K
written_k = paged_kv_cache[actual_page_id, 0, within_page_offset]
original_k = k_append[0]

print(f"🔍 验证信息:")
print(f"  - 序列位置 (position): {actual_pos}")
print(f"  - 页号 (page offset): {page_offset}")
print(f"  - 页内偏移 (within-page offset): {within_page_offset}")
print(f"  - 物理页 ID: {actual_page_id.item()}")
print(f"  - written_k shape: {written_k.shape}")
print(f"  - original_k shape: {original_k.shape}")

# 验证写入是否正确
if torch.allclose(written_k, original_k, atol=1e-3):
    print("\n✅ K 写入正确！")
else:
    print("\n❌ K 写入错误")
    print(f"Max diff: {(written_k - original_k).abs().max().item()}")

🔍 验证信息:
  - 序列位置 (position): 4095
  - 页号 (page offset): 255
  - 页内偏移 (within-page offset): 15
  - 物理页 ID: 8231
  - written_k shape: torch.Size([4, 128])
  - original_k shape: torch.Size([4, 128])

✅ K 写入正确！

✅ K 写入正确！


In [ ]:
# 📊 显存详细统计
print("=" * 60)
print("显存占用详情:")
print("=" * 60)
print(f"paged_kv_cache 理论大小: {paged_kv_cache.element_size() * paged_kv_cache.numel() / 1024**3:.4f} GB")
print(f"all_indices 大小: {all_indices.element_size() * all_indices.numel() / 1024**2:.2f} MB")
print(f"kv_page_indices 大小: {kv_page_indices.element_size() * kv_page_indices.numel() / 1024:.2f} KB")
print(f"\nCUDA 显存统计:")
print(f"  - allocated (实际使用): {torch.cuda.memory_allocated() / 1024**3:.4f} GB")
print(f"  - reserved (预留池): {torch.cuda.memory_reserved() / 1024**3:.4f} GB")
print(f"  - max allocated (峰值): {torch.cuda.max_memory_allocated() / 1024**3:.4f} GB")
print("=" * 60)

显存占用详情:
paged_kv_cache 理论大小: 1.8311 GB
all_indices 大小: 0.23 MB
kv_page_indices 大小: 1.00 KB

CUDA 显存统计:
  - allocated (实际使用): 1.8323 GB
  - reserved (预留池): 1.8359 GB
  - max allocated (峰值): 1.8341 GB


: 